In [3]:
#pip install selenium
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.chrome.service import Service

In [4]:
#pip install --upgrade selenium

In [5]:
chrome_options = webdriver.ChromeOptions()
# Add any additional options you need

service = Service(executable_path="C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")

# Specify the binary location (replace with your Chrome executable path)
chrome_options.binary_location = "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe"

# Launch the driver
driver = webdriver.Chrome(service=service, options=chrome_options)

In [6]:
# get user input for routes
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")

Please enter -1 when done.
----------
From which city?
MAD
Where to?
DUB
----------
From which city?
DUB
Where to?
MAD
----------
From which city?
-1

Routes:
MAD => DUB
DUB => MAD


In [7]:
# get user input for period (start and end date)
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
days = end_date - start_date
num_days = days.item().days

Start Date, Please use YYYY-MM-DD format only 2024-03-01
End Date, Please use YYYY-MM-DD format only 2024-03-03


In [8]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('span',class_='codeshares-airline-names',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('div',class_='section stops')

    for i in stops:
        for j in i.find_all('span',class_='stops-text'):
               stops_list.append(j.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='Flights-Results-FlightPriceSection right-alignment sleek')

    for i in price:
        for j in i.find_all('span', class_='price-text'):
            prices.append(j.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='section duration allow-multi-modal-icons')
    for i in duration:
        for j in i.find_all('div',class_='top'):
            duration_list.append(j.text)
    return duration_list

In [12]:
# Set the path to the ChromeDriver executable
chromedriver_path = "C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# Launch the driver using the Service class
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination", "Duration", "Total stops", "Price", "Date"]
    df = pd.DataFrame(columns=column_names)

    for j in tqdm(range(num_days + 1)):
        # close and open driver every 10 days to avoid captcha
        if j % 10 == 0:
            driver.quit()
            service = Service(chromedriver_path)
            driver = webdriver.Chrome(service=service)

        url = f"https://www.en.kayak.sa/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        driver.get(url)
        sleep(15)

        # click show more button to get all flights
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//a[@class="moreButton"]'))
            )
            show_more_button.click()
        except NoSuchElementException:
            print("Show more button not found. Skipping to the next iteration.")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)
        total_stops = get_total_stops(soup)
        prices = get_price(soup)
        duration = get_duration(soup)
        df = df.append(pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops': total_stops,
            'Price': prices,
            'Date': start_date + j
        }))

    df['Source'] = sources[i]
    df['Destination'] = destinations[i]
    df = df.replace('\n', '', regex=True)
    df = df.reset_index(drop=True)

    # save data as a CSV file for each route
    df.to_csv(f'{sources[i]}_{destinations[i]}.csv', index=False)
    print(f"Successfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv")

driver.quit()

  0%|          | 0/3 [00:34<?, ?it/s]


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7211482B2+55298]
	(No symbol) [0x00007FF7210B5E02]
	(No symbol) [0x00007FF720F705AB]
	(No symbol) [0x00007FF720FB175C]
	(No symbol) [0x00007FF720FB18DC]
	(No symbol) [0x00007FF720FECBC7]
	(No symbol) [0x00007FF720FD20EF]
	(No symbol) [0x00007FF720FEAAA4]
	(No symbol) [0x00007FF720FD1E83]
	(No symbol) [0x00007FF720FA670A]
	(No symbol) [0x00007FF720FA7964]
	GetHandleVerifier [0x00007FF7214C0AAB+3694587]
	GetHandleVerifier [0x00007FF72151728E+4048862]
	GetHandleVerifier [0x00007FF72150F173+4015811]
	GetHandleVerifier [0x00007FF7211E47D6+695590]
	(No symbol) [0x00007FF7210C0CE8]
	(No symbol) [0x00007FF7210BCF34]
	(No symbol) [0x00007FF7210BD062]
	(No symbol) [0x00007FF7210AD3A3]
	BaseThreadInitThunk [0x00007FFA41347614+20]
	RtlUserThreadStart [0x00007FFA42EA26A1+33]
